In [ ]:
# Comparing the accuracies of each frequency band using NN models

In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
from scipy import signal
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle as pkl
import itertools 
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedShuffleSplit

%matplotlib inline 
# %matplotlib qt


In [2]:
# Import filtered datasets 

# 0.05s 
#alpha
t_alpha005 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.05s\\alpha/think.pkl")
r_alpha005 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.05s\\alpha/read.pkl")
#theta
t_theta005 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.05s\\theta/think.pkl")
r_theta005 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.05s\\theta/read.pkl")

# 0.1s

#alpha
t_alpha01 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.1s\\alpha/think.pkl")
r_alpha01 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.1s\\alpha/read.pkl")
#theta
t_theta01 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.1s\\theta/think.pkl")
r_theta01 = pd.read_pickle("F:\EEG-data\\think-read\\filtered\\0.1s\\theta/read.pkl")

print(t_alpha01.shape)
print(r_alpha01.iloc[0,-1])

(320, 32)
[-1025.9706589   -977.37886969  -696.16757795  -248.60606733
   259.14171583   706.21097741   985.85916395  1031.01818106
   830.43051956   431.46849735   -71.07012045  -557.52661941]


In [3]:
# Shape each dataframe into No_samples,sampLen(6 or 12),32
def reshape_df(df):
    sampLen = len(df.iloc[0,0])
    N = df.shape[0]
    new_df = np.zeros((N,sampLen,32))
    for i in range(32):
        channel = df.iloc[:,i].values 
        channel_df = np.zeros((N,sampLen))
        for j in range(len(channel)):
            channel_df[j,:] = channel[j]
        new_df[:,:,i] = keras.utils.normalize(channel_df)
    return new_df


In [4]:
# 0.05s 

#alpha
t_alpha005r = reshape_df(t_alpha005)
r_alpha005r = reshape_df(r_alpha005)
#theta
t_theta005r = reshape_df(t_theta005)
r_theta005r = reshape_df(r_theta005)

# 0.1s

#alpha
t_alpha01r = reshape_df(t_alpha01)
r_alpha01r = reshape_df(r_alpha01)
#theta
t_theta01r = reshape_df(t_theta01)
r_theta01r = reshape_df(r_theta01)


In [5]:
print(t_alpha005r.shape)
print(r_alpha005r.shape)

print(t_theta005r.shape)
print(r_theta005r.shape)

print(t_alpha01r.shape)
print(r_alpha01r.shape)

print(t_theta01r.shape)
print(r_theta01r.shape)


(640, 6, 32)
(640, 6, 32)
(640, 6, 32)
(640, 6, 32)
(320, 12, 32)
(320, 12, 32)
(320, 12, 32)
(320, 12, 32)


In [6]:
#Combine theta and alpha bands 
#0.05s
t_ta005 = np.concatenate((t_alpha005r,t_theta005r),axis=2)
r_ta005 = np.concatenate((r_alpha005r,r_theta005r),axis=2)
#0.1s
t_ta01 = np.concatenate((t_alpha01r,t_theta01r),axis=2)
r_ta01 = np.concatenate((r_alpha01r,r_theta01r),axis=2)

print(r_ta005.shape)
print(r_ta01.shape)

(640, 6, 64)
(320, 12, 64)


In [18]:
# Concat test
#0.05s
t_ta005t = np.concatenate((t_alpha005r,t_theta005r),axis=1)
r_ta005t = np.concatenate((r_alpha005r,r_theta005r),axis=1)
#0.1s
t_ta01t = np.concatenate((t_alpha01r,t_theta01r),axis=1)
r_ta01t = np.concatenate((r_alpha01r,r_theta01r),axis=1)

print(t_ta005t.shape)
print(t_ta01t.shape)

(640, 12, 32)
(320, 24, 32)


In [7]:
#0.05
X005 = np.vstack((t_ta005,r_ta005))
y005 = np.hstack((np.zeros(t_ta005.shape[0]),np.ones(r_ta005.shape[0])))
print(X005.shape)
print(y005.shape)
#0.1
X01 = np.vstack((t_ta01,r_ta01))
y01 = np.hstack((np.zeros(t_ta01.shape[0]),np.ones(r_ta01.shape[0])))
print(X01.shape)
print(y01.shape)

(1280, 6, 64)
(1280,)
(640, 12, 64)
(640,)


In [19]:
#CONCAT TEST
#0.05
X005t = np.vstack((t_ta005t,r_ta005t))
y005t = np.hstack((np.zeros(t_ta005t.shape[0]),np.ones(r_ta005t.shape[0])))
print(X005t.shape)
print(y005t.shape)
#0.1
X01t = np.vstack((t_ta01t,r_ta01t))
y01t = np.hstack((np.zeros(t_ta01t.shape[0]),np.ones(r_ta01t.shape[0])))
print(X01t.shape)
print(y01t.shape)

(1280, 12, 32)
(1280,)
(640, 24, 32)
(640,)


In [8]:
#Split into train and test sets
def split_train_test(X,y):
    sss = StratifiedShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for train_index, test_index in sss.split(X,y):
                x_train, x_test = X[train_index],X[test_index]
                y_train, y_test = y[train_index],y[test_index]
    return x_train,x_test,y_train,y_test

In [9]:
#0.05s
x_train005,x_test005,y_train005,y_test005 = split_train_test(X005,y005)
#0.1s
x_train01,x_test01,y_train01,y_test01 = split_train_test(X01,y01)

In [20]:
#0.05s
x_train005t,x_test005t,y_train005t,y_test005t = split_train_test(X005t,y005t)
#0.1s
x_train01t,x_test01t,y_train01t,y_test01t = split_train_test(X01t,y01t)

In [10]:
#DNN model
def DNN_model(sampLen,no_bands):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(sampLen*32*no_bands, input_shape = (sampLen,32*no_bands), activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"), 
        tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy",
                optimizer='adam',
                metrics=["accuracy"])
    # model.summary()
    return model

#Conv model
def CONV_model(sampLen,no_bands):
    model2 = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=128, kernel_size=5,strides=1, padding="causal",
                                activation="relu",input_shape=(sampLen,32*no_bands)), #input shape = (n_timesteps,n_features)
        tf.keras.layers.Conv1D(filters=64, kernel_size=5,strides=1, padding="causal",activation="relu"),
        tf.keras.layers.Conv1D(filters=32, kernel_size=5,strides=1, padding="causal",activation="relu"),
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.MaxPool1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=16, kernel_size=5,strides=1, padding="causal",activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="relu"), 
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    # optimizer = tf.keras.optimizers.Adam(lr=9e-4)
    model2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    return model2

#Conv + LSTM
def CONV_LSTM_model(sampLen,no_bands):
    model3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=5,strides=1, padding="causal",
                              activation="relu",input_shape=(sampLen,32*no_bands)), #input shape = (n_timesteps,n_features)
    tf.keras.layers.Conv1D(filters=64, kernel_size=5,strides=1, padding="causal",activation="relu"),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5,strides=1, padding="causal",activation="relu"),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPool1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=16, kernel_size=5,strides=1, padding="causal",activation="relu"),
    # tf.keras.layers.Flatten(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dense(10, activation="relu"), 
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
    model3.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])
    return model3

#Simple RNN 
def RNN_model(sampLen,no_bands):
    model4 = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(40, input_shape = (sampLen,32*no_bands),return_sequences=True),
    tf.keras.layers.SimpleRNN(40),
    # tf.keras.layers.Dense(10,activation="relu",input_shape=(12,32)),
    tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    model4.compile(loss="binary_crossentropy",
                optimizer='adam',
                metrics=["accuracy"])
    return model4

#LSTM only

def LSTM_model(sampLen,no_bands):
    model5 = tf.keras.models.Sequential([
    # tf.keras.layers.Flatten(input_shape=(sampLen,32)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True,input_shape=(sampLen,32*no_bands))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    model5.compile(loss="binary_crossentropy",
                optimizer='adam',
                metrics=["accuracy"])
    return model5

In [11]:
def test(model,dataset,devset,batchsize,epochs):
    tf.random.set_seed(10)
    datasetb = dataset.batch(batchsize)
    devsetb = devset.batch(batchsize)
    history = model.fit(datasetb,epochs=epochs,verbose=0,validation_data=devsetb)
    return history 

In [12]:
# 0.05s 
dataset005 = tf.data.Dataset.from_tensor_slices((x_train005,y_train005))
devset005  = tf.data.Dataset.from_tensor_slices((x_test005,y_test005))

# 0.1s 
dataset01 = tf.data.Dataset.from_tensor_slices((x_train01,y_train01))
devset01  = tf.data.Dataset.from_tensor_slices((x_test01,y_test01))

In [22]:
# 0.05s 
dataset005t = tf.data.Dataset.from_tensor_slices((x_train005t,y_train005t))
devset005t  = tf.data.Dataset.from_tensor_slices((x_test005t,y_test005t))

# 0.1s 
dataset01t = tf.data.Dataset.from_tensor_slices((x_train01t,y_train01t))
devset01t  = tf.data.Dataset.from_tensor_slices((x_test01t,y_test01t))

In [13]:
# 0.05s 
# Testing different models, epochs=10, batchsize = 1 
sampLen = 6 
no_bands = 2
batchsize = 1
DNN_model005 = DNN_model(sampLen,no_bands)
CONV_model005 = CONV_model(sampLen,no_bands)
CONV_LSTM_model005 = CONV_LSTM_model(sampLen,no_bands)
LSTM_model005 = LSTM_model(sampLen,no_bands)
RNN_model005 = RNN_model(sampLen,no_bands)
#Histories 
DNN_hist005 = test(DNN_model005,dataset005,devset005,batchsize,10)
CONV_hist005 = test(CONV_model005,dataset005,devset005,batchsize,10)
CONV_LSTM_hist005 = test(CONV_LSTM_model005,dataset005,devset005,batchsize,10)
LSTM_hist005 = test(LSTM_model005,dataset005,devset005,batchsize,10)
RNN_hist005 = test(RNN_model005,dataset005,devset005,batchsize,10)
#Max validation accuracy 
valacc_DNN005 = max(DNN_hist005.history['val_accuracy'])
valacc_CONV005 = max(CONV_hist005.history['val_accuracy'])
valacc_CONVLSTM005 = max(CONV_LSTM_hist005.history['val_accuracy'])
valacc_LSTM005 = max(LSTM_hist005.history['val_accuracy'])
valacc_RNN005 = max(RNN_hist005.history['val_accuracy'])

print(valacc_DNN005)
print(valacc_CONV005)
print(valacc_CONVLSTM005)
print(valacc_LSTM005)
print(valacc_RNN005)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0.7011718153953552
0.62890625
0.6015625
0.69921875
0.55078125


In [14]:
# 0.1s 
# Testing different models, epochs=10, batchsize = 1 
sampLen = 12
no_bands = 2
batchsize = 1
DNN_model01 = DNN_model(sampLen,no_bands)
CONV_model01 = CONV_model(sampLen,no_bands)
CONV_LSTM_model01 = CONV_LSTM_model(sampLen,no_bands)
LSTM_model01 = LSTM_model(sampLen,no_bands)
RNN_model01 = RNN_model(sampLen,no_bands)
#Histories 
DNN_hist01 = test(DNN_model01,dataset01,devset01,batchsize,10)
CONV_hist01 = test(CONV_model01,dataset01,devset01,batchsize,10)
CONV_LSTM_hist01 = test(CONV_LSTM_model01,dataset01,devset01,batchsize,10)
LSTM_hist01 = test(LSTM_model01,dataset01,devset01,batchsize,10)
RNN_hist01 = test(RNN_model01,dataset01,devset01,batchsize,10)
#Max validation accuracy 
valacc_DNN01 = max(DNN_hist01.history['val_accuracy'])
valacc_CONV01 = max(CONV_hist01.history['val_accuracy'])
valacc_CONVLSTM01 = max(CONV_LSTM_hist01.history['val_accuracy'])
valacc_LSTM01 = max(LSTM_hist01.history['val_accuracy'])
valacc_RNN01 = max(RNN_hist01.history['val_accuracy'])

print(valacc_DNN01)
print(valacc_CONV01)
print(valacc_CONVLSTM01)
print(valacc_LSTM01)
print(valacc_RNN01)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0.6445311307907104
0.625
0.5
0.625
0.5390625


In [25]:
##CONCAT TEST
# 0.05s 
# Testing different models, epochs=10, batchsize = 1 
sampLen = 6*2 
no_bands = 1
batchsize = 1
DNN_model005 = DNN_model(sampLen,no_bands)
CONV_model005 = CONV_model(sampLen,no_bands)
CONV_LSTM_model005 = CONV_LSTM_model(sampLen,no_bands)
LSTM_model005 = LSTM_model(sampLen,no_bands)
RNN_model005 = RNN_model(sampLen,no_bands)
#Histories 
DNN_hist005 = test(DNN_model005,dataset005t,devset005t,batchsize,10)
CONV_hist005 = test(CONV_model005,dataset005t,devset005t,batchsize,10)
CONV_LSTM_hist005 = test(CONV_LSTM_model005,dataset005t,devset005t,batchsize,10)
LSTM_hist005 = test(LSTM_model005,dataset005t,devset005t,batchsize,10)
RNN_hist005 = test(RNN_model005,dataset005t,devset005t,batchsize,10)
#Max validation accuracy 
valacc_DNN005 = max(DNN_hist005.history['val_accuracy'])
valacc_CONV005 = max(CONV_hist005.history['val_accuracy'])
valacc_CONVLSTM005 = max(CONV_LSTM_hist005.history['val_accuracy'])
valacc_LSTM005 = max(LSTM_hist005.history['val_accuracy'])
valacc_RNN005 = max(RNN_hist005.history['val_accuracy'])

print(valacc_DNN005)
print(valacc_CONV005)
print(valacc_CONVLSTM005)
print(valacc_LSTM005)
print(valacc_RNN005)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0.6751301884651184
0.73046875
0.5
0.73828125
0.5859375
